In [ ]:
import networkx as nx
import random
import copy
import time
import matplotlib.pyplot as plt
import sys
import ndlib.models.ModelConfig as mc
import ndlib.models.epidemics.SIModel as si
import ndlib.models.opinions.VoterModel as vt
from collections import defaultdict
%matplotlib tk

def change_ds(spl_gen, num_nodes, max_radius):
    ''' 
    Transforms 'spl_gen' into the accepted datastructure
    by the 'infected_average_by_separation_degree' method
    '''
    result = []
    for n in range(num_nodes):
        di_tmp = {}
        for entry in spl_gen:  
            for radius in range(1, max_radius + 1):
                li_tmp = []
                for node, dist in entry[1].items():
                    if dist == radius:
                        li_tmp.append(node)
                if li_tmp != []:
                    di_tmp[radius - 1] = li_tmp
            result.append(copy.copy(di_tmp))
            
    return result

def infected_percentage(nodes, infected) :
    count = 0
    for node in nodes :
        if node in infected :
            count += 1
    
    return count / len(nodes)

def infected_average_by_separation_degree(G, separation_degrees, infected) :
    node_count = {}
    percentage_sum = {}
    
    for node in range(0, len(separation_degrees)) :
        if node in infected :
            for degree in separation_degrees[node] :
                if degree not in percentage_sum.keys() :
                    percentage_sum[degree] = 0
                    node_count[degree] = 0
                percentage_sum[degree] += infected_percentage(separation_degrees[node][degree], infected)
                node_count[degree] += 1
        
    average_by_separation_degree = {}
    
    for separation_degree in percentage_sum :
        average_by_separation_degree[separation_degree] = percentage_sum[separation_degree] / node_count[separation_degree]
            
    return average_by_separation_degree

def simulate_random_infecteds(G_rand, num_nodes, radius_split_rand, num_infects):
    random_infected = []
    random_not_infected = list(range(0, num_nodes))
    
    for _ in range(num_infects):
        new_inf = random.choice(random_not_infected)
        random_infected.append(new_inf)
        random_not_infected.remove(new_inf)

    avg_trait_sharing_by_radius = infected_average_by_separation_degree(G_rand, separation_degrees, random_infected)

    return (avg_trait_sharing_by_radius, random_infected)

def propensity_by_radius(radius, avg_traits, rand_avg_traits):
    return (avg_traits[radius-1] / rand_avg_traits[radius-1]) - 1

def all_propensities(max_radius, avg_traits, rand_avg_traits):
    all_propensities = []
    for r in range(1, max_radius + 1):
        all_propensities.append(propensity_by_radius(r, avg_traits, rand_avg_traits))
    return all_propensities

def normalized_propensities(max_radius, all_propensities):
    normalized_propensities = []
    for r in range(max_radius):
        normalized_propensities.append(all_propensities[r] / all_propensities[0])
    
    return normalized_propensities



In [ ]:
NUM_NODES = 1000
MAX_RADIUS_TO_CHECK = 6

results_propen = []
results_deg = []
results_other = []

for itt in range(0, 30) :
    print('Doing iteration', itt + 1)
    G = nx.barabasi_albert_graph(NUM_NODES, 3)
    
    results_other.append([2 * G.number_of_edges() / G.number_of_nodes(),nx.average_clustering(G),nx.average_shortest_path_length(G)])

    spl = nx.all_pairs_shortest_path_length(G)
    separation_degrees = change_ds(spl, NUM_NODES, MAX_RADIUS_TO_CHECK)
    # Model selection
    model = si.SIModel(G)
    #model = vt.VoterModel(G)

    # Model Configuration
    cfg = mc.Configuration()
    cfg.add_model_parameter('beta', 0.05) # probability of infection
    cfg.add_model_parameter("percentage_infected", 0) # initially infected % (if 0 => 1 infected)
    model.set_initial_status(cfg)
    start = time.time()

    infected = []
    it_count = 0
    while len(infected) < NUM_NODES / 2 :
        it_count += 1
        iteration = model.iteration()
        for node, i in iteration['status'].items() :
            if i == 1:
                infected.append(node)


    avg_traits = infected_average_by_separation_degree(G, separation_degrees, infected)
    infected_count = len(infected)

    avg_random_trait_sharing_by_radius, final_random_infected = simulate_random_infecteds(G, NUM_NODES, separation_degrees, infected_count)

    max_sep_deg = -1
    for node in separation_degrees:
        if max_sep_deg < len(node):
            max_sep_deg = len(node)

    all_props = all_propensities(max_sep_deg, avg_traits, avg_random_trait_sharing_by_radius) 
    results_propen.append(normalized_propensities(max_sep_deg, all_props))

    total_nodes_per_sep_deg = [0 for _ in range(max_sep_deg)]
    for node in separation_degrees :
        for dist in node:
            total_nodes_per_sep_deg[dist] += len(node[dist])
            
    avg_nodes_per_sep_deg = []
    for total_nodes in total_nodes_per_sep_deg:
        avg_nodes_per_sep_deg.append(total_nodes/len(separation_degrees))

    results_deg.append(avg_nodes_per_sep_deg)

In [ ]:
print(results_propen)

In [ ]:
print(results_deg)

In [ ]:
print(results_other)